In [1]:
import os
import logging
import argparse

from tqdm import tqdm
import geopandas as gpd
from owslib.wms import WebMapService

import os 
os.chdir('../')
from utils import data_utils
from utils import config_utils

%load_ext autoreload
%autoreload 2

In [82]:
name = "issa"
config_file = os.path.join(os.getcwd(), "configs/sat_configs/sat_config_500x500_60cm-temp.yaml")
creds_file = os.path.join(os.getcwd(), f"configs/sat_configs/{name}_sat_creds.yaml")
config = config_utils.load_config(config_file)
creds = config_utils.create_config(creds_file)

In [83]:
data = gpd.read_file("./data/vectors/GIGAv2/school/unicef/MNG_unicef.geojson")
data = data_utils.convert_crs(data, data.crs, "EPSG:3857") #config["srs"])
data.sample(3)

,UID,source,iso,country,region,subregion,name,giga_id_school,lon,lat,geometry
677,UNICEF-MNG-SCHOOL-00000677,UNICEF,MNG,Mongolia,Asia,Eastern Asia,None,677,1.190195e+07,6.093582e+06,POINT (11901950.223 6093582.299)
190,UNICEF-MNG-SCHOOL-00000190,UNICEF,MNG,Mongolia,Asia,Eastern Asia,None,190,1.226019e+07,5.608038e+06,POINT (12260187.644 5608038.495)
548,UNICEF-MNG-SCHOOL-00000548,UNICEF,MNG,Mongolia,Asia,Eastern Asia,None,548,1.190308e+07,6.094762e+06,POINT (11903077.245 6094762.235)


In [84]:
import pandas as pd
index = 0
image_file = f"{name}.tiff"

url = f"{config['digitalglobe_url']}connectid={creds['connect_id']}"
wms = WebMapService(url, username=creds["username"], password=creds["password"])

# Define bounding box for the image request
bbox = {
    "lon" : [
        data.lon[index] - config["size"],
        data.lon[index] + config["size"]
    ],
    "lat": [
        data.lat[index] - config["size"],
        data.lat[index] + config["size"]
    ]
}
bbox = pd.DataFrame(bbox)
bbox = gpd.GeoDataFrame(bbox, geometry = gpd.points_from_xy(
        x = bbox.lon,
        y = bbox.lat,
        crs = 'EPSG:3857', 
))
bbox

,lon,lat,geometry
0,1.320441e+07,6.045870e+06,POINT (13204406.937 6045869.790)
1,1.320471e+07,6.046170e+06,POINT (13204706.937 6046169.790)


In [85]:
bbox = bbox.to_crs("EPSG:4326")
bbox.geometry.x[0]

118.61720568869491

In [86]:

print(bbox.crs)
bbox = (
    bbox.geometry.x[0],
    bbox.geometry.y[0],
    bbox.geometry.x[1],
    bbox.geometry.y[1]
)
bbox

EPSG:4326


(118.61720568869491, 47.63212189942154, 118.61990063454726, 47.63393796028878)

In [87]:
# Request image from WMS
img = wms.getmap(
    bbox=bbox,
    layers=config["layers"],
    srs=config["srs"],
    size=(config["width"], config["height"]),
    featureProfile=config["featureprofile"],
    coverage_cql_filter=config["coverage_cql_filter"],
    exceptions=config["exceptions"],
    transparent=config["transparent"],
    format=config["format"],
)
# Save the image to file
with open(image_file, "wb") as file:
    file.write(img.read())

In [3]:
!gdalwarp -t_srs EPSG:3857 issa.tiff issa.tiff

/bin/bash: gdalwarp: command not found
